In [ ]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.framework import ops
import matplotlib.image as mpimg
import os

%matplotlib inline
np.random.seed(1)
tf.compat.v1.disable_eager_execution()

Using TensorFlow backend.


In [2]:
img = mpimg.imread('test/000e7aa6-100b-4c6b-9ff0-e7a8e53e4465_5L.png')
imgplot = plt.imshow(img)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'test/000e7aa6-100b-4c6b-9ff0-e7a8e53e4465_5L.png'

In [7]:
image = Image.open('test/000e7aa6-100b-4c6b-9ff0-e7a8e53e4465_5L.png')
print(image.format)
print(image.size)
print(image.mode)
# show the image
print(image)

PNG
(128, 128)
L
<PIL.PngImagePlugin.PngImageFile image mode=L size=128x128 at 0x15199720400>


In [7]:
data = np.asarray(image)
print(data.shape)
print(data)
batches = []
Y = []
count = 0

(128, 128)
[[71 70 70 ... 53 54 55]
 [70 71 71 ... 51 53 54]
 [70 71 73 ... 47 49 50]
 ...
 [45 43 41 ... 65 66 67]
 [47 44 39 ... 62 65 66]
 [48 45 37 ... 61 64 65]]


In [9]:
length = -1
for root, dirs, files in os.walk('train'):
    batch = []
    y = []
    length = len(files)
    for i in range(length):
        if i != 0 and i % 1000 == 0:
            batches.append(np.array(batch))
            batch = []
            y = np.array(y)
            Y.append(y)
            y = []
        image = Image.open('train/' + files[i])
        data = np.asarray(image)
        data = data.reshape((data.shape[0], data.shape[1], 1))
        batch.append(data)
        val = int(files[i][-6])
        answer = []
        
        for j in range(6):
            if j == val:
                answer.append(1)
            else:
                answer.append(0)
        
        y.append(answer)
    Y.append(y)
    if len(batch) != 0:
        batches.append(batch)
print("Done")
print("No. of training examples:", length)

Done
No. of training examples: 18000


In [8]:
length = -1
test_x = []
test_y = []
for root, dirs, files in os.walk('test'):
    y = []
    length = len(files)
    for i in range(length):
        image = Image.open('test/' + files[i])
        data = np.asarray(image)
        data = data.reshape((data.shape[0], data.shape[1], 1))
        test_x.append(data)
        val = int(files[i][-6])
        answer = []
        
        for j in range(6):
            if j == val:
                answer.append(1)
            else:
                answer.append(0)
        test_y.append(answer)
print("Done")
print("No. of testing examples:", length)

Done
No. of testing examples: 3600


In [10]:
print(len(test_y))

3600


In [11]:
print(len(batches))
print(len(Y))
print(Y[0])

18
18
[[1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 1 0 0 0]
 ...
 [1 0 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]]


In [12]:
print(batches[2][0].shape)
print(batches[2][0][0])

(128, 128, 1)
[[33]
 [38]
 [47]
 [49]
 [42]
 [39]
 [42]
 [45]
 [48]
 [47]
 [42]
 [42]
 [47]
 [50]
 [50]
 [50]
 [49]
 [49]
 [50]
 [52]
 [56]
 [55]
 [48]
 [45]
 [45]
 [46]
 [49]
 [48]
 [43]
 [40]
 [38]
 [40]
 [47]
 [47]
 [41]
 [41]
 [45]
 [46]
 [44]
 [44]
 [46]
 [44]
 [37]
 [36]
 [42]
 [44]
 [43]
 [42]
 [43]
 [45]
 [50]
 [53]
 [53]
 [50]
 [45]
 [41]
 [38]
 [37]
 [39]
 [42]
 [45]
 [46]
 [46]
 [45]
 [43]
 [41]
 [39]
 [38]
 [39]
 [41]
 [45]
 [46]
 [44]
 [40]
 [33]
 [32]
 [36]
 [40]
 [46]
 [46]
 [39]
 [34]
 [33]
 [33]
 [35]
 [41]
 [50]
 [53]
 [50]
 [49]
 [48]
 [46]
 [43]
 [43]
 [48]
 [50]
 [48]
 [45]
 [41]
 [38]
 [36]
 [36]
 [37]
 [36]
 [34]
 [34]
 [35]
 [38]
 [43]
 [43]
 [37]
 [37]
 [43]
 [43]
 [39]
 [37]
 [37]
 [37]
 [41]
 [41]
 [38]
 [34]
 [31]
 [28]
 [27]
 [25]
 [19]
 [17]]


In [45]:
inputs = layers.Input((128, 128, 1))
x = layers.Conv2D(3, 4, padding = 'same', activation = 'relu')(inputs)
x = layers.MaxPooling2D(pool_size = (3, 3))(x)
x = layers.Conv2D(32, 3, padding = 'valid', activation = 'relu')(x)
x = layers.MaxPooling2D(pool_size = (3, 3))(x)
x = layers.Conv2D(64, 3, padding = 'valid', activation = 'relu')(x)
x = layers.MaxPooling2D(pool_size = (2,2))(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation = 'relu')(x)
x = layers.Dense(64, activation = 'relu')(x)
x = layers.Dense(32, activation = 'relu')(x)
outputs = []
for i in range(6):
    outputs.append(layers.Dense(16, activation = 'sigmoid')(x))

model = keras.Model(inputs = inputs, outputs = outputs)

In [46]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(lr = 0.5),
    metrics = ['accuracy']
)

In [13]:
X_test = [i for i in batches[-1]]
Y_test = []
X_train = []
Y_train = []
print(len(Y))

for i in range(len(batches)):
    for j in batches[i]:
        X_train.append(j)
    
    for j in Y[i]:
        Y_train.append(j)

for i in Y[-1]:
    Y_test.append(i)
print(len(X_train))
X_train = np.array(X_train)
#X_test = np.array(X_test)
X_test = np.array(test_x)
Y_train = list(np.array(Y_train).T)
Y_test = list(np.array(test_y).T)
for i in range(len(Y_train)):
    Y_train[i] = Y_train[i].T
    Y_test[i] = Y_test[i].T
#Y_test = np.array(Y_test)
print(X_train.shape)
print(X_test.shape)
print(len(Y_train))
print(Y_train[0].shape)
print(len(Y_test))
print(Y_test[0].shape)

18
18000
(18000, 128, 128, 1)
(3600, 128, 128, 1)
6
(18000,)
6
(3600,)


In [48]:
print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 128, 128, 3)  51          input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_10 (MaxPooling2D) (None, 42, 42, 3)    0           conv2d_11[0][0]                  
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 40, 40, 32)   896         max_pooling2d_10[0][0]           
____________________________________________________________________________________________

In [49]:
for i in model.layers:
    print(i.output_shape)

[(None, 128, 128, 1)]
(None, 128, 128, 3)
(None, 42, 42, 3)
(None, 40, 40, 32)
(None, 13, 13, 32)
(None, 11, 11, 64)
(None, 5, 5, 64)
(None, 1600)
(None, 128)
(None, 64)
(None, 32)
(None, 16)
(None, 16)
(None, 16)
(None, 16)
(None, 16)
(None, 16)


In [50]:
model.fit(X_train, Y_train, batch_size = 1000, epochs = 10, verbose = 2)

Train on 18000 samples
Epoch 1/10
18000/18000 - 127s - loss: 13428.4628 - dense_30_loss: 1474.3491 - dense_31_loss: 2846.1155 - dense_32_loss: 606.1018 - dense_33_loss: 3365.4751 - dense_34_loss: 3326.2075 - dense_35_loss: 1810.2147 - dense_30_accuracy: 0.6033 - dense_31_accuracy: 0.7126 - dense_32_accuracy: 0.5634 - dense_33_accuracy: 0.6017 - dense_34_accuracy: 0.6034 - dense_35_accuracy: 0.6032
Epoch 2/10
18000/18000 - 116s - loss: 5.9615 - dense_30_loss: 1.0454 - dense_31_loss: 0.4745 - dense_32_loss: 1.2068 - dense_33_loss: 1.0211 - dense_34_loss: 1.1634 - dense_35_loss: 1.0504 - dense_30_accuracy: 0.8333 - dense_31_accuracy: 0.8333 - dense_32_accuracy: 0.8333 - dense_33_accuracy: 0.8333 - dense_34_accuracy: 0.8333 - dense_35_accuracy: 0.8333
Epoch 3/10
18000/18000 - 113s - loss: 3.0889 - dense_30_loss: 0.5153 - dense_31_loss: 0.4557 - dense_32_loss: 0.5580 - dense_33_loss: 0.5077 - dense_34_loss: 0.5424 - dense_35_loss: 0.5098 - dense_30_accuracy: 0.8333 - dense_31_accuracy: 0.83

In [51]:
model.evaluate(X_test, Y_test, batch_size = 1000, verbose = 2)

C:\Users\Narendra Shetty\Documents\Tanish\programming\anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


[2.7119023270077176,
 0.45119497,
 0.4542778,
 0.44958326,
 0.45303404,
 0.45474315,
 0.44869536,
 0.8333333,
 0.8333333,
 0.8333333,
 0.8333333,
 0.8333333,
 0.8333333]

In [52]:
model.save_weights('saved_weights/') #Saving it in a folder sharing the same directory as your ipython file.

In [53]:
model.save('complete_model')

INFO:tensorflow:Assets written to: complete_model\assets


In [54]:
model.save_weights('Neural-Network/with_tensorflow/handimages/saved_weights/') #Saving it for github

In [55]:
model.save('Neural-Network/with_tensorflow/handimages/complete_model/')

INFO:tensorflow:Assets written to: Neural-Network/with_tensorflow/handimages/complete_model/assets


In [11]:
model1 = tf.keras.models.load_model('complete_model/')

In [12]:
model1.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 128, 128, 3)  51          input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_10 (MaxPooling2D) (None, 42, 42, 3)    0           conv2d_11[0][0]                  
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 40, 40, 32)   896         max_pooling2d_10[0][0]           
____________________________________________________________________________________________

In [13]:
model1.evaluate(X_test, Y_test, batch_size = 1000, verbose = 2)

NameError: name 'X_test' is not defined